<a href="https://colab.research.google.com/github/Tomyao/CSE258/blob/master/Visit_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import gzip
from collections import defaultdict
from google.colab import files

In [0]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [0]:
### Upload files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

<IPython.core.display.Javascript object>

User uploaded file "pairs_Visit.txt" with length 880029 bytes


In [0]:
### Save files

for fn in uploaded.keys():
  newFile = open(fn, "wb")
  newFileByteArray = bytearray(uploaded[fn])
  newFile.write(newFileByteArray)

In [0]:
### Check files are in directory

from os import listdir
from os.path import isfile, join
mypath = "./"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f)) and not f.startswith('.')]
for myfiles in onlyfiles:
  print (myfiles)

train.json.gz
pairs_Visit.txt
predictions_Visit.txt


In [0]:
### Read in data and split into training and validation sets

allData = []

for l in readGz("train.json.gz"):
  allData.append(l)

print len(allData)
  
allData_training = allData[:len(allData)/2]
allData_validation = allData[len(allData)/2:]


200000


In [0]:
blah = []
for l in open("pairs_Visit.txt"):
  blah.append(l)

In [0]:
print len(blah)

40001


In [0]:
import numpy as np

def jaccard_similarities(mat):
    cols_sum = mat.getnnz(axis=0)
    ab = mat.T * mat

    # for rows
    aa = np.repeat(cols_sum, ab.getnnz(axis=0))
    # for columns
    bb = cols_sum[ab.indices]

    similarities = ab.copy()
    similarities.data /= (aa + bb - ab.data)

    return similarities

In [0]:
import numpy as np
from scipy import sparse

bIDtoindex = {}
indextobID = {}
uIDtoindex = {}
indextouID = {}

uIDtobID = {}

### Build jaccard scores
def build_similar(my_samples):
  # get unique businesses
  temp = set(d['businessID'] for d in my_samples)
  index = 0
  for val in temp:
    bIDtoindex[val] = index
    indextobID[index] = val
    index += 1
    
  # get unique users
  temp = set(d['userID'] for d in my_samples)
  index = 0
  for val in temp:
    uIDtoindex[val] = index
    indextouID[index] = val
    uIDtobID[val] = []
    index += 1
  print index
  
  # build user to business maps
  for d in my_samples:
    u = d['userID']
    b = d['businessID']
    uIDtobID[u].append(b)
  
  # build sparse matrix
  mymatrix = np.zeros((len(bIDtoindex), len(uIDtoindex)))
  for d in my_samples:
    u = d['userID']
    b = d['businessID']
    x = uIDtoindex[u]
    y = bIDtoindex[b]
    mymatrix[y][x] = 1

  mysparse = sparse.csc_matrix(mymatrix)
    
  return jaccard_similarities(mysparse)

In [0]:
test = build_similar(allData)


18793


In [0]:
indices = zip(*test.nonzero())

In [0]:
test2 = test.todense()

In [0]:
print test2.item(0, 0)

1.0


In [0]:
indexmap = {}
for i,j in indices:
  if i not in indexmap:
    indexmap[i] = [j]
  else:
    indexmap[i].append(j)

In [0]:
print indexmap[0][0]

18659


In [0]:
print indextobID

{0: 'B758492670', 1: 'B216830637', 2: 'B109727892', 3: 'B807454692', 4: 'B079183309', 5: 'B707595844', 6: 'B424552244', 7: 'B873107816', 8: 'B775239177', 9: 'B523319373', 10: 'B364305397', 11: 'B381694879', 12: 'B690263576', 13: 'B197227925', 14: 'B264089981', 15: 'B055568155', 16: 'B805213437', 17: 'B414377060', 18: 'B446238268', 19: 'B671685877', 20: 'B027884087', 21: 'B813225482', 22: 'B683180592', 23: 'B717920069', 24: 'B628265643', 25: 'B039113709', 26: 'B880239182', 27: 'B686153636', 28: 'B799459509', 29: 'B607776382', 30: 'B798443367', 31: 'B255533529', 32: 'B922664643', 33: 'B534912182', 34: 'B621239672', 35: 'B237576295', 36: 'B195137241', 37: 'B308085029', 38: 'B308112546', 39: 'B469785862', 40: 'B959481218', 41: 'B582438385', 42: 'B335215004', 43: 'B738688496', 44: 'B250729927', 45: 'B982345785', 46: 'B300050828', 47: 'B508345297', 48: 'B876160793', 49: 'B829388381', 50: 'B182256997', 51: 'B870118422', 52: 'B834998872', 53: 'B965009823', 54: 'B059929599', 55: 'B163236469', 5

In [0]:
predictions = open("predictions_Visit.txt", 'w')
mydata = []

blah = 0

for l in open("pairs_Visit.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,b = l.strip().split('-')
  # see if a similar user went to the business
  if u in uIDtoindex and b in bIDtoindex:
    uindex = uIDtoindex[u]
    visited = False
    for otheruser2 in indexmap[uindex]:
      
      if test2.item(uindex,otheruser2) > 0.001:
        
        otheruserid = indextouID[otheruser2]
        if b in uIDtobID[otheruserid]:
          visited = True
          break;
    if visited:
      predictions.write(u + '-' + b + ",1\n")
    else:
      predictions.write(u + '-' + b + ",0\n")
  else:
    predictions.write(u + '-' + b + ",0\n")
  
  blah += 1
  if blah % 10000 == 0:
    print blah
      
print "done!"

10000
20000
30000
40000
done!


In [0]:
files.download('predictions_Visit.txt')